In [1]:
import os
from bs4 import BeautifulSoup, NavigableString, Tag 

from nltk.util import ngrams
from nltk import word_tokenize

import json
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as plt
import seaborn as sns

import collections
import csv

from datetime import datetime
from pytz import timezone

print "imported"

imported


In [2]:
# IMPORTS SPECIFIC TO TOPIC MODELING

import langid
import nltk
import re
import time
from collections import defaultdict
from configparser import ConfigParser
from gensim import corpora, models, similarities
from nltk.tokenize import RegexpTokenizer
from string import digits
print "imported"

imported


In [213]:
files = [filename.split('.')[0] for filename in os.listdir(inpath) if not filename.startswith('.')]
print files

OSError: [Errno 2] No such file or directory: 'digitalSTS/papers/txt/02_Parmiggiani_Monteiro.txt03_Allhutter.txt'

### Loading Documents and Splitting Into Lines

In [256]:
#LOAD DOCUMENT BY LINE INTO AN ARRAY


files = [filename.split('.')[0] for filename in os.listdir('digitalSTS/papers/txt/') if not filename.startswith('.')]
# print files

for filename in files:

    inpath = 'digitalSTS/papers/txt/' + filename + '.txt'
    outpath = 'digitalSTS/papers/lines/' + filename + '.csv'
    
    with open(inpath, 'r') as infile, open(outpath, 'w') as outfile:
        
#         for line in infile.readlines():
#             print line
        paragraphs = infile.readlines()
    
        for paragraph in paragraphs:

            paragraph = paragraph.decode('unicode_escape').encode('ascii','ignore')
            lines = [line.strip().lower() for line in ''.join(paragraph).split('.')]
            #             print paragraph
            
            writer = csv.writer(outfile)

            for line in lines:
                words = line.split(' ')
                if len(words) > 1:
                    writer.writerow([line.encode('utf-8')])

In [277]:
files = [filename.split('.')[0] for filename in os.listdir('digitalSTS/papers/csvs/') if not filename.startswith('.')]
print files

['02_Parmiggiani_Monteiro', '03_Allhutter', '04_Camus_Vinck', '05_Lynch', '06_Vertesi', '07_Kerisadou', '08_Couture', '09_Dunbar_Hester', '10_Hawthorne', '11_Chan', '12_Poster', '13_Ilten_McInerney', '14_Nemer', '15_Erikson', '16_Toth', '17_Cohn', '18_Seaver', '19_Stark', '20_Ribes', '21_Cardoso_Llach', '22_Venturini', '23_Salamanca', '24_Munk', '25_Calvillo', '26_Winthereik', '27_Loukissas']


### Creating Dictionaries and Corpuses

In [279]:
files = [filename.split('.')[0] for filename in os.listdir('digitalSTS/papers/csvs/') if not filename.startswith('.')]
# print files

for filename in files:

    inpath = 'digitalSTS/papers/csvs/' + filename + '.csv'
    
    with open(inpath, 'r') as infile:
        
        reader = csv.reader(infile)
        documents = [line[0] for line in reader]
#         print lines

        # TOKENIZING
        tokenizer = RegexpTokenizer(r'\w+')
        documents = [tokenizer.tokenize(doc.lower()) for doc in documents ]
        
        # GATHERING CANDIDATES FOR CUSTOM LIST OF STOPWORDS FROM NGRAMS
        # ['i','m','a','a','i','t','6',...]
        unigrams = [ w for doc in documents for w in doc if len(w)==1]
        # ['do','or','or','of','be',...]
        bigrams  = [ w for doc in documents for w in doc if len(w)==2]

        misc = ['get', 'know', 'would']
        
        # CREATES THE SET OF UNIQUE "CUSTOM STOP LIST WITH NLTK + THE ABOVE
        stoplist  = set(nltk.corpus.stopwords.words("english") + unigrams + bigrams + misc)

        # REMOVES STOP WORDS
        documents = [[token for token in doc if token not in stoplist]
                        for doc in documents]

        # REMOVES WORDS THAT ARE NUMBERS ONLY
        documents = [ [token for token in doc if len(token.strip(digits)) == len(token)]
                        for doc in documents ]
        
        # REMOVE WORDS THAT OCCUR ONLY ONCE
        token_frequency = defaultdict(int)

        # count all tokens
        for doc in documents:
            for token in doc:
                token_frequency[token] += 1

        # keep words that occur more than once
        documents = [ [token for token in doc if token_frequency[token] > 1]
                        for doc in documents  ]

        # Sort words in documents
        for doc in documents:
            doc.sort()
            
        # I HATE UNICODE!!
        documents = [ [token.decode('ascii', 'ignore') for token in doc] for doc in documents  ]
        
        # Build a dictionary where for each document each word has its own id
        dictionary = corpora.Dictionary(documents)
        dictionary.compactify()
        # and save the dictionary for future use
        print dictionary
        dictionary.save( 'digitalSTS/papers/dicts/' + filename + '.dict')
        
        # BUILD AND SAVE CORPUS
        corpus = [dictionary.doc2bow(doc) for doc in documents]
        corpora.MmCorpus.serialize('digitalSTS/papers/corpuses/' + filename + '.mm', corpus)

Dictionary(873 unique tokens: [u'represent', u'concept', u'managed', u'forbidden', u'lack']...)
Dictionary(821 unique tokens: [u'represent', u'concept', u'semantic', u'grassroots', u'scratch']...)
Dictionary(948 unique tokens: [u'concept', u'semantic', u'consider', u'magnetic', u'dynamic']...)
Dictionary(784 unique tokens: [u'limited', u'code', u'forbidden', u'lack', u'focus']...)
Dictionary(955 unique tokens: [u'represent', u'todays', u'consider', u'focus', u'yellow']...)
Dictionary(782 unique tokens: [u'concept', u'managed', u'focus', u'four', u'woods']...)
Dictionary(644 unique tokens: [u'limited', u'code', u'consider', u'chain', u'focus']...)
Dictionary(698 unique tokens: [u'limited', u'code', u'illustrates', u'hooks', u'oss']...)
Dictionary(772 unique tokens: [u'represent', u'concept', u'consider', u'limited', u'dissolution']...)
Dictionary(794 unique tokens: [u'limited', u'code', u'managed', u'global', u'dynamic']...)
Dictionary(1245 unique tokens: [u'represent', u'concept', u'pa

In [266]:
# CODE HELP FROM:
# https://github.com/alexperrier/datatalks/blob/master/twitter/twitter_preprocessing.py

# SPLITS EACH POST INTO WORDS WHILE REMOVING PUNCTUATION (\w+) AND LOWER CASES THEM A WORD SUCH AS
# I'M WILL BECOME I, M
tokenizer = RegexpTokenizer(r'\w+')
documents = [tokenizer.tokenize(doc.lower()) for doc in documents ]
## [['what', 'do', 'they', 'look', 'for', 'for', 'approval', 'or', 'denial'],
##  ['hi','i','m','wondering','if','someone','could','answer',...]...
documents[:2]

[['digitized', 'coral', 'reefs'],
 ['elena', 'parmiggiani', 'and', 'eric', 'monteiro']]

In [267]:
# I'M NOT SURE THESE ARE TECHNICALLY UNIGRAMS AND BIGRAMS SO MUCH AS THEY ARE SINGLE LETTER WORDS
# AND TWO LETTER WORDS BUT WHO AM I TO JUDEGE (SHULDER SHURG)

# ['i','m','a','a','i','t','6',...]
unigrams = [ w for doc in documents for w in doc if len(w)==1]
# ['do','or','or','of','be',...]
bigrams  = [ w for doc in documents for w in doc if len(w)==2]

misc = ['get', 'know', 'would']

print unigrams[:5]
print bigrams[:5]

['a', 'a', 'a', 'a', 'a']
['mr', 'in', 'is', 'in', 'on']


In [268]:
# CREATES THE SET OF UNIQUE "CUSTOM STOP LIST WITH NLTK + THE ABOVE
stoplist  = set(nltk.corpus.stopwords.words("english") + unigrams + bigrams + misc)

# REMOVES STOP WORDS
documents = [[token for token in doc if token not in stoplist]
                for doc in documents]

# REMOVES WORDS THAT ARE NUMBERS ONLY
documents = [ [token for token in doc if len(token.strip(digits)) == len(token)]
                for doc in documents ]

# # [['look', 'approval', 'denial', 'parole', 'approval'...],
# # ['wondering', 'someone', 'could', 'answer', 'question'...]...]
print documents[0][:5]
print documents[1][:5]

['digitized', 'coral', 'reefs']
['elena', 'parmiggiani', 'eric', 'monteiro']


In [269]:
# REMOVE WORDS THAT OCCUR ONLY ONCE
token_frequency = defaultdict(int)

# count all tokens
for doc in documents:
    for token in doc:
        token_frequency[token] += 1

# keep words that occur more than once
documents = [ [token for token in doc if token_frequency[token] > 1]
                for doc in documents  ]

# Sort words in documents
for doc in documents:
    doc.sort()
    
## [['approval', 'approval', 'approval', 'approval', 'approval'...],
##  ['answer', 'benefit', 'benefit', 'benefit', 'confused'...]...]

print documents[0][:5]
print documents[1][:5]

['coral', 'digitized', 'reefs']
['monteiro', 'parmiggiani']


In [270]:
# I HATE UNICODE!!
documents = [ [token.decode('ascii', 'ignore') for token in doc] for doc in documents  ]
print documents[0][:5]
print documents[1][:5]

[u'coral', u'digitized', u'reefs']
[u'monteiro', u'parmiggiani']


In [271]:
# Build a dictionary where for each document each word has its own id
dictionary = corpora.Dictionary(documents)
dictionary.compactify()
# and save the dictionary for future use
dictionary.save('parmiggiani.dict')

In [272]:
# We now have a dictionary with 11127 unique tokens
# Dictionary(11127 unique tokens: [u'foul', u'narcotic', u'four', u'woods', u'hanging']...)
print dictionary

Dictionary(873 unique tokens: [u'represent', u'concept', u'managed', u'forbidden', u'lack']...)


In [273]:
# Build the corpus: vectors with occurence of each word for each document
# convert tokenized documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in documents]

In [274]:
# and save in Market Matrix format
corpora.MmCorpus.serialize('parmiggiani.mm', corpus)
# this corpus can be loaded with corpus = corpora.MmCorpus('prison_talk_nyc.mm')